# Problem Statement

The ComputeCloud company offers eight cloud services across five data centers: Alpha, Beta, 
Gamma, Delta, and Epsilon. These data centers can host various types of services: Web Hosting 
(WH), Database Services (DB), File Storage (FS), Data Analytics (DA), Virtual Private Servers 
(VPS), Content Delivery Network (CDN), Machine Learning (ML), and Internet of Things 
(IoT) services. Each data center has limitations on computational power (measured in CPU
GHz), storage capacity (measured in Terabytes - TB), and network bandwidth (measured in 
Gbps). Each type of service has different requirements for computational power, storage, and 
bandwidth, as well as different operational costs and revenue potentials. ComputeCloud needs 
to decide how much of each service to run in each data center to maximize their profit. and 
ensuring that customer demand and service level agreements (SLAs) are met.

**Data Center Capacities**

| Resource                          | Alpha | Beta | Gamma | Delta | Epsilon |
| --------------------------------- | ----- | ---- | ----- | ----- | ------- |
| Computational Power (CPU-GHz)     | 2500  | 2000 | 2300  | 3000  | 1500    |
| Storage Capacity (Terabytes - TB) | 1200  | 2500 | 1800  | 3500  | 1000    |
| Network Bandwidth (Gbps)          | 800   | 1500 | 1000  | 900   | 600     |


**Service Requirements (per unit)**

| Metric                            | WH  | DB  | FS  | DA  | VPS | CDN | ML  | IoT |
| --------------------------------- | --- | --- | --- | --- | --- | --- | --- | --- |
| Computational Power (CPU-GHz)     | 8   | 12  | 4   | 18  | 10  | 5   | 20  | 6   |
| Storage Capacity (Terabytes - TB) | 4   | 8   | 15  | 12  | 6   | 3   | 10  | 2   |
| Network Bandwidth (Gbps)          | 2   | 4   | 2   | 8   | 3   | 10  | 5   | 1   |
| Service Profit (per unit) in \$   | 90  | 140 | 70  | 190 | 110 | 160 | 210 | 120 |
| Minimum Demand (units)            | 120 | 90  | 160 | 75  | 130 | 85  | 60  | 140 |


**Profit per unit**

| Service | Profit per Unit (\$) |
| ------- | -------------------- |
| WH      | 90                   |
| DB      | 140                  |
| FS      | 70                   |
| DA      | 190                  |
| VPS     | 110                  |
| CDN     | 160                  |
| ML      | 210                  |
| IoT     | 120                  |


* **Independent service constraints**( Some service may require others to function) - Each unit of Database Services (DB) requires at least half a unit of File Storage (FS)

* **Shared resource constraints**(Some services share a common infrastructure) - Web Hosting (WH) and Content Delivery Network (CDN) services share a common 
load balancer in each center having capacity of 300 units.


* **Data sovereignty compliance**-IoT data must be stored and processed within the Alpha or Beta centers only due to data sovereignty laws.





# Optimization Model



 **Decision Variables**

Let:

$$
x_{ij} = \text{Number of units of service } i \text{ allocated in data center } j
$$

Where:

* $i \in \{ \text{WH, DB, FS, DA, VPS, CDN, ML, IoT} \}$
* $j \in \{ \text{Alpha, Beta, Gamma, Delta, Epsilon} \}$

---

 **Objective Function (Maximize Profit)**

$$
\text{Maximize } Z = \sum_{i,j} \text{Profit}_i \cdot x_{ij}
$$


---

**Subject to:**

1. **Resource Constraints per Data Center**

For each data center $j$:

**a. CPU Constraint:**

$$
\sum_i \text{CPU}_i \cdot x_{ij} \leq \text{CPU Capacity}_j
$$

**b. Storage Constraint:**

$$
\sum_i \text{Storage}_i \cdot x_{ij} \leq \text{Storage Capacity}_j
$$

**c. Bandwidth Constraint:**

$$
\sum_i \text{Bandwidth}_i \cdot x_{ij} \leq \text{Bandwidth Capacity}_j
$$

 2. **Minimum SLA Demand Constraints**

For each service $i$:

$$
\sum_j x_{ij} \geq \text{Minimum Demand}_i
$$


---
 **Interdependencies & Constraints**

 a. **Database Requires File Storage**

$$
\sum_j x_{\text{DB},j} \leq 0.5 \cdot \sum_j x_{\text{FS},j}
$$

 b. **WH + CDN Share Load Balancer**

For each data center $j$:

$$
x_{\text{WH},j} + x_{\text{CDN},j} \leq 300
$$

c. **Data Sovereignty for IoT**

$$
x_{\text{IoT},j} = 0 \quad \forall j \notin \{\text{Alpha, Beta}\}
$$

---

 **Non-Negativity**

$$
x_{ij} \geq 0 \quad \forall i,j
$$

---


In [1]:
from scipy.optimize import linprog
import pandas as pd

# Define services and data centers
services = ["WH", "DB", "FS", "DA", "VPS", "CDN", "ML", "IoT"]
data_centers = ["Alpha", "Beta", "Gamma", "Delta", "Epsilon"]

# Profit per unit
profit = {"WH": 90, "DB": 140, "FS": 70, "DA": 190, "VPS": 110, "CDN": 160, "ML": 210, "IoT": 120}

# Resource requirements per service
cpu = {"WH": 8, "DB": 12, "FS": 4, "DA": 18, "VPS": 10, "CDN": 5, "ML": 20, "IoT": 6}
storage = {"WH": 4, "DB": 8, "FS": 15, "DA": 12, "VPS": 6, "CDN": 3, "ML": 10, "IoT": 2}
bandwidth = {"WH": 2, "DB": 4, "FS": 2, "DA": 8, "VPS": 3, "CDN": 10, "ML": 5, "IoT": 1}
demand = {"WH": 120, "DB": 90, "FS": 160, "DA": 75, "VPS": 130, "CDN": 85, "ML": 60, "IoT": 140}

# Resource capacities per data center
cpu_cap = {"Alpha": 2500, "Beta": 2000, "Gamma": 2300, "Delta": 3000, "Epsilon": 1500}
storage_cap = {"Alpha": 1200, "Beta": 2500, "Gamma": 1800, "Delta": 3500, "Epsilon": 1000}
bandwidth_cap = {"Alpha": 800, "Beta": 1500, "Gamma": 1000, "Delta": 900, "Epsilon": 600}

# Index mapping for variables
index = {(s, d): i * len(data_centers) + j for i, s in enumerate(services) for j, d in enumerate(data_centers)}

# Objective function: maximize profit => minimize negative profit
c = [-profit[s] for s in services for d in data_centers]

# Constraints
A_ub, b_ub = [], []

# Resource constraints per data center
for d in data_centers:
    A_ub.append([cpu[s] if d2 == d else 0 for s in services for d2 in data_centers])
    b_ub.append(cpu_cap[d])
    A_ub.append([storage[s] if d2 == d else 0 for s in services for d2 in data_centers])
    b_ub.append(storage_cap[d])
    A_ub.append([bandwidth[s] if d2 == d else 0 for s in services for d2 in data_centers])
    b_ub.append(bandwidth_cap[d])
    A_ub.append([1 if s in ["WH", "CDN"] and d2 == d else 0 for s in services for d2 in data_centers])
    b_ub.append(300)  # load balancer capacity

# DB must be ≤ 0.5 × FS globally
row = [1 if s == "DB" else -0.5 if s == "FS" else 0 for s in services for d in data_centers]
A_ub.append(row)
b_ub.append(0)

# SLA demand constraints (≥) — convert to -sum(x) ≤ -demand
for s in services:
    row = [-1 if s2 == s else 0 for s2 in services for d in data_centers]
    A_ub.append(row)
    b_ub.append(-demand[s])

# Bounds: IoT only in Alpha and Beta
bounds = []
for s in services:
    for d in data_centers:
        if s == "IoT" and d not in ["Alpha", "Beta"]:
            bounds.append((0, 0))
        else:
            bounds.append((0, None))

# Solve LP
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

# Extract and display results
if res.success:
    print(f"\n✅ Optimal Profit: ${-res.fun:.2f}\n")
    print("📊 Optimal Service Allocation:")
    for (s, d), idx in index.items():
        val = res.x[idx]
        if val > 1e-3:
            print(f"{s:5} in {d:8}: {val:.2f} units")
else:
    print("❌ Optimization failed:", res.message)



✅ Optimal Profit: $181520.53

📊 Optimal Service Allocation:
WH    in Delta   : 133.50 units
DB    in Gamma   : 40.00 units
DB    in Epsilon : 50.00 units
FS    in Beta    : 9.84 units
FS    in Gamma   : 28.41 units
FS    in Delta   : 141.75 units
DA    in Gamma   : 25.00 units
DA    in Epsilon : 50.00 units
VPS   in Gamma   : 113.50 units
VPS   in Delta   : 16.50 units
CDN   in Alpha   : 41.82 units
CDN   in Beta    : 125.84 units
CDN   in Gamma   : 24.27 units
ML    in Delta   : 60.00 units
IoT   in Alpha   : 381.82 units
IoT   in Beta    : 221.91 units
